In [1]:
import dspy
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, MIPRO
from dspy.evaluate import Evaluate
from dspy.teleprompt.ensemble import Ensemble
from datasets import load_dataset
import random
import re
import os

# save all our hard work in the prompts directory
os.makedirs('prompts', exist_ok=True)

random.seed(0)

COMPILE_FS = False
COMPILE_MIPRO = False
COMPILE_COT = False

/home/toolkit/.conda/envs/dspy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Why or why not dspy?

The goal of dspy is to extract better performance out of LLMs using prompting.

* Pros:
    * Write code instead of prompts.
    * Data and metric (reward) driven.
    * Write the code once and run with multiple LLMs.
* Cons: 
    * You need a big context size for all this prompting.
    * Nothing is free, you need a lot of calls to evaluate all these prompts. (ideally you would use a batch job and not a notebook)
    * Still in early development, there is refactoring and sometimes the interface is inconsistent.


In [2]:
lm = dspy.HFClientVLLM(model="meta-llama/Meta-Llama-3-8B-Instruct",
                       port=8080,
                       url="http://localhost")
dspy.settings.configure(lm=lm)
NUM_THREADS = 8

# Dataset

* Create a dataset using the dspy.Example class
* We will use the Ultra Feedback dataset
    * 1 instruction
    * 4 possible completions
    * all of them are rated by gpt4
* Our goal is to find a good prompt to get the best RM out of Llama3
* We will create a train, valid, and test dataset
* We will evaluate our model using exact match

In [3]:
dataset = load_dataset("openbmb/UltraFeedback")
all_data = []
for input_ in dataset["train"]:
    completions = sorted(
        input_["completions"], key=lambda x: x["overall_score"], reverse=True
    )
    # only take the top and bottom completions
    for i, chosen_completion in enumerate(completions[:1]):
        for rejected_completion in completions[-1:]:
            if (
                chosen_completion["overall_score"]
                == rejected_completion["overall_score"]
            ):
                continue
            if random.random() < 0.5:
                text1 = chosen_completion["response"]
                text2 = rejected_completion["response"]
                preference = "1"
            else:
                text1 = rejected_completion["response"]
                text2 = chosen_completion["response"]
                preference = "2"
                
            if text1 == text2:
                continue
            # llama 3 has a smart context window
            if len(text1) > 1524 or len(text2) > 1524 or len(input_["instruction"]) > 1524:
                continue
            
            all_data.append(
                dspy.Example(
                    **{
                        "instruction": input_["instruction"],
                        "text_1": text1,
                        "text_2": text2,
                        "preferred_text": preference,
                    }
                ).with_inputs("instruction", "text_1", "text_2")
            )
    if len(all_data) > 2000:
        break

random.shuffle(all_data)

valid = all_data[:100]
test = all_data[100:600]
train = all_data[600:]

/home/toolkit/.conda/envs/dspy/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [4]:
def extract_pref(example, pred):
    pref_pred = re.search(r"\d+", pred["preferred_text"])
    if pref_pred:
        pref_pred = pref_pred.group()
    else:
        pref_pred = None
    return pref_pred


def em_metric(example, pred, trace=None, frac=1.0, verbose=False):
    pref_pred = extract_pref(example, pred)
    if verbose:
        print(f"Example: {example['preferred_text']}")
        print(f"Prediction: {pref_pred}")
    score = example["preferred_text"] == pref_pred
    if score is None:
        return False
    return score

eval_fn = Evaluate(devset=test, metric=em_metric, num_threads=NUM_THREADS, display_progress=True)

In [5]:
class Preference(dspy.Signature):
    instruction = dspy.InputField()
    text_1 = dspy.InputField()
    text_2 = dspy.InputField()
    preferred_text = dspy.OutputField(desc="Only return the preferred text (1 or 2) as an int", prefix="preferred_text:")

class PrefPredict(dspy.Module):
    def __init__(self):
        super().__init__()
        self.preference = dspy.Predict(
            Preference, max_tokens=3, temperature=0.1
        ) 

    def forward(self, instruction, text_1, text_2, *args, **kwargs):
        preferred = self.preference(
                        instruction=instruction,
            text_1=text_1,
            text_2=text_2,
        )
        return preferred

In [6]:
pred = PrefPredict()(**train[0])
print(pred)
em_metric(train[0], pred, verbose=True)

Prediction(
    preferred_text='1'
)
Example: 1
Prediction: 1


True

In [7]:
lm.inspect_history(n=1)





Given the fields `instruction`, `text_1`, `text_2`, produce the fields `preferred_text`.

---

Follow the following format.

Instruction: ${instruction}

Text 1: ${text_1}

Text 2: ${text_2}

preferred_text: Only return the preferred text (1 or 2) as an int

---

Instruction: For a given set of 10 food-related words, provide the frequency of their usage in a corpus of reviews and the average number of syllables for each word. Furthermore, can you identify the top 3 food-related words with the highest frequency of usage and calculate the standard deviation of their syllable counts across the corpus?

Text 1: Here are the frequencies of the ten food-related words in a corpus of restaurant reviews, along with the average number of syllables: - Pasta: 188 occurrences, 1.5 syllables - Salad: 157 occurrences, 1.7 syllables - Bread: 128 occurrences, 1.5 syllables - Chicken: 154 occurrences, 2.1 syllables - Rice: 115 occurrences, 1.5 syllables - Fish: 121 occurrences, 2.0 syllables - Meat:

In [8]:
eval_fn(PrefPredict())

Average Metric: 324 / 500  (64.8): 100%|██████████| 500/500 [00:00<00:00, 867.75it/s]


Average Metric: 324 / 500  (64.8%)


64.8

# Bootstrapping Few Shot Example with Random Search

* Need a train and a valid set, a metric, and a LLM.
* Does not require demonstration, e.g., no demo CoT will be required.
* The LLM will create good traces that satisfy the metric.
* We will use random search to find the best prompt over generated traces and input/output pairs.

In [9]:
boot_fs = BootstrapFewShotWithRandomSearch(metric=em_metric, max_bootstrapped_demos=4, max_labeled_demos=4,
                                           num_threads=NUM_THREADS, max_rounds=1, num_candidate_programs=50)

if COMPILE_FS:
    preference_model = boot_fs.compile(PrefPredict(), trainset=train, valset=valid)
    ensemble_preference_model = [prog for *_, prog in preference_model.candidate_programs[:3]]
    for idx, prog in enumerate([x[-1] for x in ensemble_preference_model]):
        prog.save(f'prompts/preference_model_{idx}.json')
else:
    ensemble_preference_model = []
    for idx in range(3):
        prog = PrefPredict()
        prog.load(f'prompts/preference_model_{idx}.json')
        ensemble_preference_model.append(prog)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 50 candidate sets.


In [10]:
eval_fn(ensemble_preference_model[0])

Average Metric: 342 / 500  (68.4): 100%|██████████| 500/500 [00:00<00:00, 774.64it/s]


Average Metric: 342 / 500  (68.4%)


68.4

In [11]:
lm.inspect_history(n=1)





Given the fields `instruction`, `text_1`, `text_2`, produce the fields `preferred_text`.

---

Follow the following format.

Instruction: ${instruction}

Text 1: ${text_1}

Text 2: ${text_2}

preferred_text: Only return the preferred text (1 or 2) as an int

---

Instruction: Take the given text and modify it to make it more interesting. It was a sunny day.

Text 1: The day was radiant with a golden hue, the sun beaming down upon the world as if it was a precious gift.

Text 2: On a bright and radiant afternoon, the sun's rays cast a warm glow over the land, bringing life and joy to all in its path. The birds sang melodies in the trees, the flowers swayed in the gentle breeze, and the grass glistened as it basked in the sun's embrace. It was a day like no other, filled with promise and potential, ready to be savored and enjoyed to its fullest.

preferred_text: 2

---

Instruction: Rewrite the following sentences with more formal and polite language. Hey, what's up?

Text 1: Greetin

# Using an ensemble

In [12]:
ensemble_optimizer = Ensemble(reduce_fn=dspy.majority)
ensemble_preference_model_fn = ensemble_optimizer.compile(ensemble_preference_model)
eval_fn(ensemble_preference_model_fn)

Average Metric: 345 / 500  (69.0): 100%|██████████| 500/500 [00:01<00:00, 314.01it/s]


Average Metric: 345 / 500  (69.0%)


69.0

# Adding Chain of Thought

In [13]:
class CoTPrefPredict(dspy.Module):
    def __init__(self):
        super().__init__()
        self.preference = dspy.ChainOfThought(
            Preference
        ) 

    def forward(self, instruction, text_1, text_2, *args, **kwargs):
        preferred = self.preference(
            instruction=instruction,
            text_1=text_1,
            text_2=text_2,
        )
        return preferred

In [14]:
if COMPILE_COT:
    cot_preference_model = boot_fs.compile(CoTPrefPredict(), trainset=train, valset=valid)
    ensemble_cot_preference_model = []
    for idx, prog in enumerate([x[-1] for x in cot_preference_model.candidate_programs[:3]]):
        prog.save(f'prompts/cot_preference_model_{idx}.json')
        ensemble_cot_preference_model.append(prog)
else:
    ensemble_cot_preference_model = []
    for idx in range(3):
        prog = CoTPrefPredict()
        prog.load(f'prompts/cot_preference_model_{idx}.json')
        ensemble_cot_preference_model.append(prog)
    

In [15]:
eval_fn(ensemble_cot_preference_model[0])

Average Metric: 352 / 500  (70.4): 100%|██████████| 500/500 [00:00<00:00, 633.95it/s]

Average Metric: 352 / 500  (70.4%)


70.4

# MIPRO (Multi-prompt Instruction Proposal Optimizer)

https://twitter.com/kristahopsalong/status/1766166198079889737

* Takes a teacher and a student LLMs, a dataset, and a metric.
* Multi stage optimization
    * The teacher looks at inputs and outputs and summarize the data
    * The student generates good traces for a few input where the metric is validated
    * The teacher create instruction given the good traces and the summary of the data
    * Use some kind of Bayes Optimization to search over instructions and examples


In [17]:
if COMPILE_MIPRO:
    mipro = MIPRO(metric=em_metric, init_temperature=1.0, num_candidates=8)
    kwargs = dict(num_threads=NUM_THREADS, display_progress=True, display_table=4)
    mipro_preference_model = mipro.compile(student=PrefPredict(), trainset=train, num_trials=20, max_bootstrapped_demos=4, 
                                            max_labeled_demos=4, eval_kwargs=kwargs, requires_permission_to_run=False)
    mipro_preference_model.save("prompts/mipro_preference_model.json")
else:
    pass



Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 1401 examples in dev set * 20 trials * # of LM calls in your program = (28020 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 8 * 1 lm calls in program = 18 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:



  1%|          | 10/1401 [00:00<00:01, 722.42it/s]


Bootstrapped 4 full traces after 11 examples in round 0.


  0%|          | 4/1401 [00:00<00:01, 703.62it/s]


Bootstrapped 4 full traces after 5 examples in round 0.


...

Average Metric: 68 / 100  (68.0%)


Average Metric: 75 / 100  (75.0): 100%|██████████| 100/100 [00:06<00:00, 15.77it/s]


Average Metric: 75 / 100  (75.0%)


Average Metric: 1 / 1  (100.0): 100%|██████████| 1/1 [00:00<00:00, 12.96it/s]
[I 2024-05-02 16:53:24,067] Trial 19 finished with value: 71.94860813704497 and parameters: {'140734164980576_predictor_instruction': 3, '140734164980576_predictor_demos': 6}. Best is trial 19 with value: 71.94860813704497.


Average Metric: 1 / 1  (100.0%)
Returning preference = Predict(StringSignature(instruction, text_1, text_2 -> preferred_text
    instructions='Produce a coherent and well-structured response by combining the provided text fragments, using the necessary punctuation and grammar to ensure clarity and readability, and tailoring the tone and language to the target audience. Use any relevant data formats or specifications to support your answer.'
    instruction = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Instruction:', 'desc': '${instruction}'})
    text_1 = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Text 1:', 'desc': '${text_1}'})
    text_2 = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Text 2:', 'desc': '${text_2}'})
    preferred_text = Field(annotation=str required=True json_schema_extra={'desc': 'Only return the preferred text (1 or

In [29]:
eval_fn(mipro_preference_model)

Average Metric: 346 / 500  (69.2): 100%|██████████| 500/500 [00:00<00:00, 556.51it/s]


Average Metric: 346 / 500  (69.2%)


69.2

In [27]:
prog = PrefPredict()
mipro_preference_model2 = prog.load('prompts/mipro_preference_model.json')

In [25]:
eval_fn(mipro_preference_model)

TypeError: BaseModule.load_state() missing 1 required positional argument: 'state'